In [47]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib tk 
import pandas as pd
import pprint
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob
import os
import xlrd
import math
import seaborn as sb
import openpyxl
#import datetime
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import plotly.plotly as py
import plotly.tools as tls

from Elia_DataReadIn import (read_elia_freq_R1, 
read_combine_elia_freq_R1, 
fetch_combine_elia_freq_R1, 
read_elia_activated_energy_volumes, 
read_elia_activated_energy_prices, 
read_combine_elia_activated_energy, 
fetch_combine_elia_activated_energy, 
read_elia_cap,
read_elia_imbalanceprices,
fetch_combine_elia_imbalanceprices)
from Elia_R2plus_activation import R2plusSingleOptimum, R2plusMultiTimeOptimum
from Elia_DataAnalysis import elia_cap_graphs, elia_activatedreserves_graphs, negativePOS, durationImbalancePrices

In [42]:
import plotly.plotly as py      # Every function in this module will communicate with an external plotly server
py.plot({                      # use `py.iplot` inside the ipython notebook
"data": [{
    "x": [1, 2, 3],
    "y": [4, 2, 5]
}],
"layout": {
    "title": "hello world"
}
}, filename='hello world',      # name of the file as saved in your plotly account
sharing='public')            # 'public' | 'private' | 'secret': Learn more: https://plot.ly/python/privacy

'https://plot.ly/~jensbaet/7'

In [146]:
dfimbalance.columns

Index(['NRV in MW', 'SI in MW', 'alpha in euro/MWh', 'MIP in euro/MWh',
       'MDP in euro/MWh', 'SR in euro/MWh', 'POS in euro/MWh',
       'NEG in euro/MWh'],
      dtype='object')

In [147]:
dfimbalance["POS in euro/MWh"].plot()

In [149]:
dfimbalance=dfimbalance["POS in euro/MWh"]

In [155]:
#heatmap, hourly by day of the week
fromdate="2018-01-01"
todate="2018-09-30"

#create mask
mask=dfimbalance.loc[(dfimbalance.index >= fromdate) & (dfimbalance.index < todate)]

#belpex price or price difference
#mask=mask.drop("Euro",axis=1)

mask=mask.groupby([mask.index.hour,mask.index.dayofweek]).mean().unstack()
plt.figure()
fig=sb.heatmap(mask,robust=True,square=False,cbar_kws={'label':'euro/MWh'})
fig.set_xlabel("day of the week")
fig.set_ylabel("hour of the day")
fig.set_xticklabels(["mon","tue","wed","thu","fri","sat","sun"])
fig.set_title('Belpex DAM prices - average weekly pattern 2017')

Text(0.5,1,'Belpex DAM prices - average weekly pattern 2017')

In [140]:
#Fetch fresh data and store as pickle format for later use

dfimbalance=fetch_combine_elia_imbalanceprices("01-2018","10-2018","all")
#dfimbalance.to_pickle("../data/ImbalancePrices_01-2016_05-2018")

2018-10-19 15:05:09.159477     amount of files to combine: 10
2018-10-19 15:05:09.159477     processing file number: 1
WARNING *** file size (581568) not 512 + multiple of sector size (512)
2018-10-19 15:05:10.890459     processing file number: 2
WARNING *** file size (540608) not 512 + multiple of sector size (512)
2018-10-19 15:05:11.720451     processing file number: 3
WARNING *** file size (581568) not 512 + multiple of sector size (512)
2018-10-19 15:05:12.601442     processing file number: 4
WARNING *** file size (565184) not 512 + multiple of sector size (512)
2018-10-19 15:05:13.461434     processing file number: 5
WARNING *** file size (581568) not 512 + multiple of sector size (512)
2018-10-19 15:05:14.301425     processing file number: 6
WARNING *** file size (573376) not 512 + multiple of sector size (512)
2018-10-19 15:05:15.151417     processing file number: 7
WARNING *** file size (589760) not 512 + multiple of sector size (512)
2018-10-19 15:05:16.121407     processing 

In [ ]:
df=pd.read_pickle("C:\\Users\\jensbaet\\Documents\\MEGAsync\\Elia_data\\data\\ActivEnergyPriceVol_01-2017_09-2018.pkl")

In [ ]:
df=pd.read_pickle("C:\\Users\\jensbaet\\Documents\\MEGAsync\\Elia_data\\data\\ImbalancePrices_01-2006_05-2018.pkl")

In [2]:
df=fetch_combine_elia_activated_energy("01-2017","10-2018","all")

2018-10-02 16:10:56.334077     amount of files to combine: 44
2018-10-02 16:10:56.334077     processing file number: 1
WARNING *** file size (663488) not 512 + multiple of sector size (512)
2018-10-02 16:10:57.284077     processing file number: 2
WARNING *** file size (589760) not 512 + multiple of sector size (512)
2018-10-02 16:10:58.199077     processing file number: 3
WARNING *** file size (647104) not 512 + multiple of sector size (512)
2018-10-02 16:10:59.169077     processing file number: 4
WARNING *** file size (630720) not 512 + multiple of sector size (512)
2018-10-02 16:11:01.679077     processing file number: 5
WARNING *** file size (647104) not 512 + multiple of sector size (512)
2018-10-02 16:11:02.629077     processing file number: 6
WARNING *** file size (630720) not 512 + multiple of sector size (512)
2018-10-02 16:11:03.529077     processing file number: 7
WARNING *** file size (655296) not 512 + multiple of sector size (512)
2018-10-02 16:11:04.472077     processing 

In [3]:
df.to_pickle("../data/ActivEnergyPriceVol_01-2017_10-2018")

In [ ]:
i = 1
fig, (ax1, ax2, ax3) = plt.subplots(3,1)

for i in range (1,5):
    
    fromdate="2018-0"+ str(i) + "-01"
    todate="2018-0" + str(i+1) + "-01"
    dfmaskimbalance=dfimbalance.loc[(dfimbalance.index >= fromdate) & (dfimbalance.index < todate)]
    
    calcneg = dfmaskimbalance.copy()
    calcneg[calcneg >= 0] = np.nan
    calcneg=calcneg["POS in euro/MWh"]

    calcpos = dfmaskimbalance.copy()
    calcpos[calcpos < 0] = np.nan
    calcpos=calcpos["POS in euro/MWh"]

    calcneg.plot(figsize=(16,6),kind="hist",ax=ax1,bins=(250),grid=True,)
    calcpos.plot(figsize=(16,6),kind="hist",ax=ax2,bins=250,grid=True)
    dfmaskimbalance["POS in euro/MWh"].plot(figsize=(16,6),kind="hist",ax=ax3,bins=500,grid=True)
    i = i+1

In [ ]:
calcneg.hist(bins=100)

In [ ]:
dfimbalance["POS in euro/MWh"].hist(bins=500)

In [ ]:
#plot the negative POS imbalance price

#dfimbalance = pd.read_pickle("../data/ImbalancePrices_01-2006_05-2018.pkl")
negativePOS(dfimbalance,"2018-01-01","2018-06-01",'M')

In [ ]:
durationImbalancePrices(dfimbalance,"2018-01-01","2018-02-01","BOTH",True)
durationImbalancePrices(dfimbalance,"2018-02-01","2018-03-01","BOTH",True)
durationImbalancePrices(dfimbalance,"2018-03-01","2018-04-01","BOTH",True)
durationImbalancePrices(dfimbalance,"2018-04-01","2018-05-01","BOTH",True)
durationImbalancePrices(dfimbalance,"2018-05-01","2018-06-01","BOTH",True)

In [ ]:
#fetch fresh data and plot negative POS imbalance price
dfimbalance=fetch_combine_elia_imbalanceprices("01-2018","06-2018","all")
negativePOS(dfimbalance,"01-01-2006","01-06-2018",'M')

In [ ]:
df.columns

In [ ]:
df["R3 Flex (€/MWh)"].dropna()

In [ ]:
#code because Elia changed the names in the excel data files... already implemented in the "fetch_combine_elia_activated_energy" method.
df.fillna(0,inplace=True)
df["Bids+ in euro/MWh"] = df["Bids+\n (€/MWh)"] + df["Bids+ in euro/MWh"]
df["Bids- in euro/MWh"] = df["Bids-\n(€/MWh)"] + df["Bids- in euro/MWh"]
df["IGCC- in euro/MWh"] = df["IGCC-\n(€/MWh)"] + df["IGCC- in euro/MWh"]
df["IGGC+ in euro/MWh"] = df["IGGC+\n(€/MWh)"] + df["IGGC+ in euro/MWh"]
df["MDP in euro/MWh"] = df["MDP\n(€/MWh)"] + df["MDP in euro/MWh"]
df["MIP in euro/MWh"] = df["MIP\n (€/MWh)"] + df["MIP in euro/MWh"]
df["R2+ in euro/MWh"] = df["R2+\n(€/MWh)"] + df["R2+ in euro/MWh"]
df["R2- in euro/MWh"] = df["R2-\n(€/MWh)"] + df["R2- in euro/MWh"]
df["R3 flex in euro/MWh"] = df["R3 Flex (€/MWh)"] + df["R3 flex in euro/MWh"]
df["R3 std in euro/MWh"] = df["R3 std (€/MWh)"] + df["R3 std in euro/MWh"]
df["R3- in euro/MWh"] = df["R3-\n(€/MWh)"] + df["R3- in euro/MWh"]
df["SR in euro/MWh"] = df["SR\n(€/MWh)"] + df["SR in euro/MWh"]
df["inter TSO import in euro/MWh"] = df["Inter-TSO Import\n (€/MWh)"] + df["inter TSO import in euro/MWh"]
df["Bids+ in MW"] = df["Bids+\n (MW)"] + df["Bids+ in MW"]
df["Bids- in MW"] = df["Bids-\n(MW)"] + df["Bids- in MW"]
df["GDV in MW"] = df["GDV\n(MW)"] + df["GDV in MW"]
df["GUV in MW"] = df["GUV\n (MW)"] + df["GUV in MW"]
df["IGCC+ in MW"] = df["IGCC+\n(MW)"] + df["IGCC+ in MW"]
df["IGCC- in MW"] = df["IGCC-\n(MW)"] + df["IGCC- in MW"]
df["inter TSO export in MW"] = df["Inter-Tso\nExport\n(MW)"] + df["inter TSO export in MW"]
df["inter TSO import in MW"] = df["Inter-Tso Import(MW)"] + df["inter TSO import in MW"]
df["NRV in MW"] = df["NRV\n (MW)"] + df["NRV in MW"]
df["R2+ in MW"] = df["R2+\n(MW)"] + df["R2+ in MW"]
df["R2- in MW"] = df["R2-\n(MW)"] + df["R2- in MW"]
df["R3 flex in MW"] = df["R3 Flex\n(MW)"] + df["R3 flex in MW"]
df["R3 std in MW"] = df["R3 Std\n (MW)"] + df["R3 std in MW"]
df["SR in MW"] = df["SR\n(MW)"] + df["SR in MW"]
df=df.drop(["Bids+\n (€/MWh)","Bids-\n(€/MWh)","IGCC-\n(€/MWh)","IGGC+\n(€/MWh)","MDP\n(€/MWh)","MIP\n (€/MWh)","R2+\n(€/MWh)","R2-\n(€/MWh)","R3 Flex (€/MWh)","R3 std (€/MWh)","R3-\n(€/MWh)","SR\n(€/MWh)","Inter-TSO Import\n (€/MWh)","Bids+\n (MW)","Bids-\n(MW)","GDV\n(MW)","GUV\n (MW)","IGCC+\n(MW)","IGCC-\n(MW)","Inter-Tso\nExport\n(MW)","Inter-Tso Import(MW)","NRV\n (MW)","R2+\n(MW)","R2-\n(MW)","R3 Flex\n(MW)","R3 Std\n (MW)","SR\n(MW)"],axis=1)
df[df == 0] = np.nan

In [ ]:
dfr3std.test[dfr3std.test == 0] = np.nan

In [ ]:
dfr3std.price1.dropna()

In [ ]:
dfr3std.test.dropna()

In [4]:
dfr3bidsplus=pd.DataFrame()
dfr3bidsplus["price"] = df["Bids+ in euro/MWh"].dropna()
dfr3bidsplus["volume"] = df["Bids+ in MW"].dropna()

In [5]:
dfr3bidsmin=pd.DataFrame()
dfr3bidsmin["price"] = df["Bids- in euro/MWh"].dropna()
dfr3bidsmin["volume"] = df["Bids- in MW"].dropna()

In [52]:
dfr3std=pd.DataFrame()
dfr3std["price"]=df["R3 std in euro/MWh"]
dfr3std["volume"]=df["R3 std in MW"]

In [7]:
dfr3flex=pd.DataFrame()
dfr3flex["price"]=df["R3 flex in euro/MWh"]
dfr3flex["volume"]=df["R3 flex in MW"]

In [112]:
#aggregate the R3 activations (bidsplus)

dfr3bidsplus=dfr3bidsplus.dropna(how="all")

c=1
c1=0
dfr3bidsplus["start"]=0
dfr3bidsplus["stop"]=0
for i in range (0,len(dfr3bidsplus)-1):
    if dfr3bidsplus.index[i] == dfr3bidsplus.index[i+1]-pd.DateOffset(minutes=15):
        if c==1:
            print("Startdate: " + str(dfr3bidsplus.index[i]))
            dfr3bidsplus["start"][[c1]]=dfr3bidsplus.index[i]
            c=2
    else:
        print ("Stopdate: " + str(dfr3bidsplus.index[i]))
        dfr3bidsplus["stop"][[c1]]=dfr3bidsplus.index[i]
        c=1
        print(str(c1) + "------------")
        c1=c1+1
print("Stopdate " + str(dfr3bidsplus.index[-1]))
dfr3bidsplus.stop[c1]=dfr3bidsplus.index[i]

for i in range (0,len(dfr3bidsplus)):
    if dfr3bidsplus["stop"][i]!=0:
        if dfr3bidsplus["start"][i] == 0:
            dfr3bidsplus["start"][i] = dfr3bidsplus["stop"][i]-pd.DateOffset(minutes=15)
            
dfr3bidsplus["delta"]=dfr3bidsplus["stop"]-dfr3bidsplus["start"]

Startdate: 2017-01-01 01:15:00
Stopdate: 2017-01-01 02:45:00
0------------
Startdate: 2017-01-01 03:15:00
Stopdate: 2017-01-01 03:45:00
1------------
Startdate: 2017-01-01 08:00:00
Stopdate: 2017-01-01 08:15:00
2------------
Startdate: 2017-01-01 13:30:00


C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Stopdate: 2017-01-01 13:45:00
3------------
Startdate: 2017-01-01 17:45:00
Stopdate: 2017-01-01 19:00:00
4------------
Startdate: 2017-01-01 21:00:00
Stopdate: 2017-01-01 23:00:00
5------------
Startdate: 2017-01-02 11:15:00
Stopdate: 2017-01-02 12:00:00
6------------
Startdate: 2017-01-02 19:15:00
Stopdate: 2017-01-02 19:30:00
7------------
Startdate: 2017-01-03 14:00:00
Stopdate: 2017-01-03 14:15:00
8------------
Startdate: 2017-01-04 00:00:00
Stopdate: 2017-01-04 00:15:00
9------------
Startdate: 2017-01-04 07:00:00
Stopdate: 2017-01-04 07:15:00
10------------
Startdate: 2017-01-04 08:45:00
Stopdate: 2017-01-04 09:00:00
11------------
Startdate: 2017-01-04 10:00:00
Stopdate: 2017-01-04 10:30:00
12------------
Startdate: 2017-01-04 11:30:00
Stopdate: 2017-01-04 12:15:00
13------------
Startdate: 2017-01-04 12:45:00
Stopdate: 2017-01-04 13:00:00
14------------
Startdate: 2017-01-04 18:00:00
Stopdate: 2017-01-04 18:30:00
15------------
Startdate: 2017-01-04 19:00:00
Stopdate: 2017-01-0

120------------
Startdate: 2017-02-10 14:45:00
Stopdate: 2017-02-10 15:00:00
121------------
Startdate: 2017-02-11 11:00:00
Stopdate: 2017-02-11 11:45:00
122------------
Startdate: 2017-02-11 12:15:00
Stopdate: 2017-02-11 13:30:00
123------------
Startdate: 2017-02-11 18:00:00
Stopdate: 2017-02-11 19:30:00
124------------
Startdate: 2017-02-12 10:45:00
Stopdate: 2017-02-12 11:00:00
125------------
Startdate: 2017-02-12 11:30:00
Stopdate: 2017-02-12 11:45:00
126------------
Startdate: 2017-02-12 12:15:00
Stopdate: 2017-02-12 13:00:00
127------------
Startdate: 2017-02-12 18:30:00
Stopdate: 2017-02-12 18:45:00
128------------
Startdate: 2017-02-14 07:00:00
Stopdate: 2017-02-14 07:15:00
129------------
Startdate: 2017-02-17 12:00:00
Stopdate: 2017-02-17 12:15:00
130------------
Stopdate: 2017-02-19 14:30:00
131------------
Startdate: 2017-02-19 21:00:00
Stopdate: 2017-02-19 21:15:00
132------------
Stopdate: 2017-02-21 20:15:00
133------------
Stopdate: 2017-02-22 17:45:00
134------------

Stopdate: 2017-05-28 21:15:00
241------------
Startdate: 2017-05-29 10:30:00
Stopdate: 2017-05-29 11:45:00
242------------
Stopdate: 2017-05-29 13:45:00
243------------
Startdate: 2017-05-29 16:00:00
Stopdate: 2017-05-29 18:00:00
244------------
Stopdate: 2017-05-29 18:30:00
245------------
Startdate: 2017-05-29 19:15:00
Stopdate: 2017-05-29 19:45:00
246------------
Stopdate: 2017-05-29 20:45:00
247------------
Startdate: 2017-05-30 00:15:00
Stopdate: 2017-05-30 00:30:00
248------------
Startdate: 2017-05-30 01:15:00
Stopdate: 2017-05-30 03:00:00
249------------
Stopdate: 2017-05-31 17:00:00
250------------
Startdate: 2017-05-31 17:30:00
Stopdate: 2017-05-31 18:00:00
251------------
Startdate: 2017-06-01 13:45:00
Stopdate: 2017-06-01 14:30:00
252------------
Stopdate: 2017-06-02 23:15:00
253------------
Startdate: 2017-06-09 08:15:00
Stopdate: 2017-06-09 09:45:00
254------------
Startdate: 2017-06-12 06:45:00
Stopdate: 2017-06-12 07:00:00
255------------
Startdate: 2017-06-14 13:15:00


...

839------------
Startdate: 2018-04-05 22:15:00
Stopdate: 2018-04-05 22:45:00
840------------
Startdate: 2018-04-05 23:15:00
Stopdate: 2018-04-05 23:45:00
841------------
Startdate: 2018-04-06 19:15:00
Stopdate: 2018-04-06 19:45:00
842------------
Startdate: 2018-04-07 00:00:00
Stopdate: 2018-04-07 01:45:00
843------------
Startdate: 2018-04-07 18:15:00
Stopdate: 2018-04-07 19:00:00
844------------
Stopdate: 2018-04-07 20:15:00
845------------
Startdate: 2018-04-07 21:30:00
Stopdate: 2018-04-07 21:45:00
846------------
Stopdate: 2018-04-08 19:15:00
847------------
Startdate: 2018-04-09 12:00:00
Stopdate: 2018-04-09 13:00:00
848------------
Startdate: 2018-04-09 15:45:00
Stopdate: 2018-04-09 16:00:00
849------------
Startdate: 2018-04-09 19:15:00
Stopdate: 2018-04-09 20:30:00
850------------
Startdate: 2018-04-10 02:15:00
Stopdate: 2018-04-10 02:30:00
851------------
Startdate: 2018-04-10 06:45:00
Stopdate: 2018-04-10 07:00:00
852------------
Startdate: 2018-04-10 16:30:00
Stopdate: 2018

Stopdate: 2018-05-23 12:00:00
955------------
Startdate: 2018-05-23 18:00:00
Stopdate: 2018-05-23 20:15:00
956------------
Startdate: 2018-05-23 21:30:00
Stopdate: 2018-05-23 22:00:00
957------------
Startdate: 2018-05-23 22:30:00
Stopdate: 2018-05-23 23:45:00
958------------
Startdate: 2018-05-24 01:30:00
Stopdate: 2018-05-24 03:00:00
959------------
Startdate: 2018-05-24 03:30:00
Stopdate: 2018-05-24 05:15:00
960------------
Startdate: 2018-05-24 09:15:00
Stopdate: 2018-05-24 09:30:00
961------------
Startdate: 2018-05-24 15:30:00
Stopdate: 2018-05-24 16:00:00
962------------
Startdate: 2018-05-24 17:45:00
Stopdate: 2018-05-24 18:15:00
963------------
Startdate: 2018-05-24 22:15:00
Stopdate: 2018-05-24 22:30:00
964------------
Startdate: 2018-05-24 23:15:00
Stopdate: 2018-05-24 23:30:00
965------------
Startdate: 2018-05-25 08:00:00
Stopdate: 2018-05-25 08:15:00
966------------
Stopdate: 2018-05-26 01:15:00
967------------
Startdate: 2018-05-26 02:30:00
Stopdate: 2018-05-26 03:30:00


1077------------
Startdate: 2018-07-01 19:30:00
Stopdate: 2018-07-01 22:00:00
1078------------
Startdate: 2018-07-02 14:30:00
Stopdate: 2018-07-02 14:45:00
1079------------
Startdate: 2018-07-02 18:00:00
Stopdate: 2018-07-02 18:15:00
1080------------
Startdate: 2018-07-02 21:15:00
Stopdate: 2018-07-02 21:45:00
1081------------
Startdate: 2018-07-02 22:45:00
Stopdate: 2018-07-02 23:45:00
1082------------
Startdate: 2018-07-03 15:15:00
Stopdate: 2018-07-03 15:30:00
1083------------
Startdate: 2018-07-03 16:15:00
Stopdate: 2018-07-03 16:30:00
1084------------
Startdate: 2018-07-03 17:15:00
Stopdate: 2018-07-03 19:15:00
1085------------
Startdate: 2018-07-03 20:15:00
Stopdate: 2018-07-03 20:30:00
1086------------
Stopdate: 2018-07-03 21:15:00
1087------------
Startdate: 2018-07-03 22:15:00
Stopdate: 2018-07-03 23:45:00
1088------------
Startdate: 2018-07-04 15:45:00
Stopdate: 2018-07-04 16:15:00
1089------------
Startdate: 2018-07-04 17:45:00
Stopdate: 2018-07-04 18:00:00
1090------------


Stopdate: 2018-08-23 09:00:00
1193------------
Startdate: 2018-08-23 10:15:00
Stopdate: 2018-08-23 10:45:00
1194------------
Startdate: 2018-08-23 13:00:00
Stopdate: 2018-08-23 13:30:00
1195------------
Stopdate: 2018-08-23 16:30:00
1196------------
Startdate: 2018-08-23 17:30:00
Stopdate: 2018-08-23 17:45:00
1197------------
Startdate: 2018-08-23 18:30:00
Stopdate: 2018-08-23 18:45:00
1198------------
Startdate: 2018-08-24 00:15:00
Stopdate: 2018-08-24 01:00:00
1199------------
Stopdate: 2018-08-24 16:15:00
1200------------
Stopdate: 2018-08-27 19:45:00
1201------------
Startdate: 2018-08-27 23:15:00
Stopdate: 2018-08-27 23:30:00
1202------------
Startdate: 2018-08-28 10:30:00
Stopdate: 2018-08-28 10:45:00
1203------------
Stopdate: 2018-08-28 12:00:00
1204------------
Stopdate: 2018-08-30 01:30:00
1205------------
Startdate: 2018-08-30 02:15:00
Stopdate: 2018-08-30 02:30:00
1206------------
Startdate: 2018-08-30 03:45:00
Stopdate: 2018-08-30 04:00:00
1207------------
Startdate: 2018-

C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [162]:
dfr3flex

,price,volume,start,stop,delta
Timestamp,,,,,
2017-01-04 19:15:00,310.00,34.0,2017-01-04 19:15:00,2017-01-04 19:30:00,0 days 00:15:00
2017-01-04 19:30:00,310.00,34.0,2017-01-25 12:15:00,2017-01-25 12:30:00,0 days 00:15:00
2017-01-25 12:15:00,NaN,60.0,2017-09-27 12:30:00,2017-09-27 12:45:00,0 days 00:15:00
2017-01-25 12:30:00,310.00,184.0,2017-09-27 13:00:00,2017-09-27 13:15:00,0 days 00:15:00
2017-09-27 12:45:00,233.32,60.0,2017-10-11 13:30:00,2017-10-11 17:00:00,0 days 03:30:00
2017-09-27 13:15:00,233.32,60.0,2017-11-29 18:30:00,2017-11-29 19:00:00,0 days 00:30:00
2017-10-11 13:30:00,121.52,78.7,2017-12-12 06:15:00,2017-12-12 06:45:00,0 days 00:30:00
2017-10-11 13:45:00,121.52,84.0,2018-01-08 17:45:00,2018-01-08 18:15:00,0 days 00:30:00
2017-10-11 14:00:00,121.52,84.0,2018-09-21 09:15:00,2018-09-21 10:30:00,0 days 01:15:00


In [111]:
#aggregate the R3 activations (flex)

dfr3flex=dfr3flex.dropna(how="all")

c=1
c1=0
dfr3flex["start"]=0
dfr3flex["stop"]=0
for i in range (0,len(dfr3flex)-1):
    if dfr3flex.index[i] == dfr3flex.index[i+1]-pd.DateOffset(minutes=15):
        if c==1:
            print("Startdate: " + str(dfr3flex.index[i]))
            dfr3flex["start"][[c1]]=dfr3flex.index[i]
            c=2
    else:
        print ("Stopdate: " + str(dfr3flex.index[i]))
        dfr3flex["stop"][[c1]]=dfr3flex.index[i]
        c=1
        print(str(c1) + "------------")
        c1=c1+1
print("Stopdate " + str(dfr3flex.index[-1]))
dfr3flex.stop[c1]=dfr3flex.index[i]

for i in range (0,len(dfr3flex)):
    if dfr3flex["stop"][i]!=0:
        if dfr3flex["start"][i] == 0:
            dfr3flex["start"][i] = dfr3flex["stop"][i]-pd.DateOffset(minutes=15)
            
dfr3flex["delta"]=dfr3flex["stop"]-dfr3flex["start"]

C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

Startdate: 2017-01-04 19:15:00
Stopdate: 2017-01-04 19:30:00
0------------
Startdate: 2017-01-25 12:15:00
Stopdate: 2017-01-25 12:30:00
1------------
Stopdate: 2017-09-27 12:45:00
2------------
Stopdate: 2017-09-27 13:15:00
3------------
Startdate: 2017-10-11 13:30:00
Stopdate: 2017-10-11 17:00:00
4------------
Startdate: 2017-11-29 18:30:00
Stopdate: 2017-11-29 19:00:00
5------------
Startdate: 2017-12-12 06:15:00
Stopdate: 2017-12-12 06:45:00
6------------
Startdate: 2018-01-08 17:45:00
Stopdate: 2018-01-08 18:15:00
7------------
Startdate: 2018-09-21 09:15:00
Stopdate 2018-09-21 10:45:00


C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [155]:
#number of hours
((dfr3std["stop"][0].to_pydatetime()-dfr3std["start"][0].to_pydatetime()).total_seconds())/3600

1.5

In [157]:
for i in len ()
dfr3std["delta"]=(dfr3std["stop"].to_pydatetime()-dfr3std["start"].to_pydatetime()).total_seconds()/3600

AttributeError: 'Series' object has no attribute 'to_pydatetime'

In [110]:
#aggregate the R3 activations (std)

dfr3std=dfr3std.dropna(how='all')

c=1
c1=0
dfr3std["start"]=0
dfr3std["stop"]=0
for i in range (0,len(dfr3std)-1):
    if dfr3std.index[i] == dfr3std.index[i+1]-pd.DateOffset(minutes=15):
        if c==1:
            print("Startdate: " + str(dfr3std.index[i]))
            dfr3std["start"][[c1]]=dfr3std.index[i]
            c=2
    else:
        print ("Stopdate: " + str(dfr3std.index[i]))
        dfr3std["stop"][[c1]]=dfr3std.index[i]
        c=1
        print(str(c1) + "------------")
        c1=c1+1
print("Stopdate " + str(dfr3std.index[-1]))
dfr3std.stop[c1]=dfr3std.index[i]

for i in range (0,len(dfr3std)):
    if dfr3std["stop"][i]!=0:
        if dfr3std["start"][i] == 0:
            dfr3std["start"][i] = dfr3std["stop"][i]-pd.DateOffset(minutes=15)
            
dfr3std["delta"]=(dfr3std["stop"].to_pydatetime()-dfr3std["start"].to_pydatetime()).total_seconds())/3600

C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

Startdate: 2017-01-01 21:15:00
Stopdate: 2017-01-01 22:45:00
0------------
Startdate: 2017-01-03 18:15:00
Stopdate: 2017-01-03 19:00:00
1------------
Startdate: 2017-01-04 19:00:00
Stopdate: 2017-01-04 20:30:00
2------------
Startdate: 2017-01-05 18:00:00
Stopdate: 2017-01-05 18:30:00
3------------
Startdate: 2017-01-06 17:30:00
Stopdate: 2017-01-06 19:30:00
4------------
Startdate: 2017-01-07 10:45:00
Stopdate: 2017-01-07 15:30:00
5------------
Stopdate: 2017-01-07 17:45:00
6------------
Startdate: 2017-01-07 19:15:00
Stopdate: 2017-01-07 20:00:00
7------------
Startdate: 2017-01-10 14:00:00
Stopdate: 2017-01-10 15:00:00
8------------
Startdate: 2017-01-10 16:15:00
Stopdate: 2017-01-10 16:30:00
9------------
Startdate: 2017-01-13 07:45:00
Stopdate: 2017-01-13 09:45:00
10------------
Startdate: 2017-01-13 17:30:00
Stopdate: 2017-01-13 18:00:00
11------------
Startdate: 2017-01-19 07:00:00
Stopdate: 2017-01-19 09:30:00
12------------
Startdate: 2017-01-23 08:45:00
Stopdate: 2017-01-23 0

C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [118]:
fig,(ax) = plt.subplots(1,1, sharex=True)
dfr3std["price"].plot(style=".",ax=ax,grid=True)
dfr3flex["price"].plot(style=".",ax=ax,grid=True)
dfr3bidsplus["price"].plot(style=".",ax=ax,grid=True)
#dfr3bidsmin["price"].plot(style=".",ax=ax,grid=True)

dfr3std["volume"].plot(style=".",ax=ax1,grid=True)
dfr3flex["volume"].plot(style=".",ax=ax1,grid=True)
dfr3bidsplus["volume"].plot(style=".",ax=ax1,grid=True)
#dfr3bidsmin["volume"].plot(style=".",ax=ax1,grid=True)

ax.set_ylabel("euro/MWh")
ax1.set_ylabel("MW")
ax.set_title("Activation prices of mFRR")
ax1.set_title("Activation volumes of mFRR")
plt.legend(["R3 standard","R3 flex","Free bids+","Free bids-"])

In [ ]:
plt.figure()
ax=sb.barplot(x=dfr3flex.index,y=dfr3flex["volume"])

In [ ]:
((df["Bids- in MW"].dropna()*df["Bids- in euro/MWh"].dropna())/4).plot()

In [ ]:
dfr3flex.reset_index()

In [ ]:
df["R3 Flex (€/MWh)"].dropna()

In [ ]:
df1=df[["R3 flex in euro/MWh"]].dropna()

In [ ]:
df1=df1.append(df[["R3 std in euro/MWh"]].dropna())

In [ ]:
df1=df1.append(df[["R3 flex in euro/MWh"]].dropna())

In [ ]:
df1=df1.sort_index()

In [ ]:
df1.loc[(df1.index >= "2018-01-01") & (df1.index < "2018-12-01")]

In [ ]:
df1.dropna(subset=["R3 flex in euro/MWh","R3 std in euro/MWh"],how="all")

In [ ]:
df["R3 flex in MW"].dropna()

In [ ]:
#read quarter hourly data on all ancillary service products, both prices and volumes
dfpricevol = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_03-2018.pkl")

In [ ]:
################### Plot graphs of activated reserves (capacity in MW, prices in euro/MWh) #####################

df = fetch_combine_elia_activated_energy("03-2018","04-2018","")
#df = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_03-2018.pkl")
producttypes = ["R2-", "Bids+", "Bids-","R2+"]
elia_activatedreserves_graphs(df,"both", "02-2018", "04-2018",producttypes)

In [ ]:
#Read 10s data on frequency and R1
dffreq = pd.read_pickle("../data/FreqData_2017.pkl")

In [ ]:
################## Plot graphs of contracted reserves (capacity in MW, prices in euro/MW/h) ########################

dfcap = read_elia_cap("../data/Ancillary services, volumes and prices.xlsx")
elia_cap_graphs(dfcap, "price", False,"01-2012","10-2018","R3")

In [ ]:
elia_cap_graphs(dfcap,"price",False,"01-2012","12-2018","R2")

In [ ]:
dfcapmask=dfcap.loc[(dfcap.index >= "2018-01-01") & (dfcap.index < "2018-12-31")]
dfcapmask=dfcapmask.loc[(dfcapmask["reserve type"] == "R3")]
dfcapmask=dfcapmask.loc[(dfcapmask["service type"] == "Flex")]
elia_cap_graphs(dfcapmask,"price",False,"01-2012","12-2018","R3")

In [ ]:
dfcapmask["average price in euro/MW/h"].mean()

In [ ]:
#53MW R2+ for whole 2017, real netto profit (taking into account gascost for extra e-production)
#daily optimum bidprice
fromtime = "2017-01-01"
totime = "2017-12-31"
bidvolume = 53
granularity = "day"

df102=R2plusMultiTimeOptimum(fromtime,totime,granularity,bidvolume,dfpricevol,dfR1R2cap,True,False)
remun102 = (int(df102.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df102.groupby(["timestamp"])[["returnCap"]].max().sum()))
PowerExtra102=df102.set_index("volumeActiv").groupby(["timestamp"])["returnAct"].idxmax().mean()*bidvolume*((pd.to_datetime(totime)+relativedelta(days=1))-(pd.to_datetime(fromtime))).days*24
bidav102=df102.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()
gascost102 = PowerExtra102 *2 * 17

print("average bidprice: " +str(float(bidav102)))
print("remuneration in €: " + str(remun102))
print("extra produced power in MWh: " +str(int(PowerExtra102)))
print("extra gascost in €: " + str(int(gascost102)))
print("real benefit in €: " + str(int(remun102-gascost102)))

In [ ]:
#20MW R2+ for whole 2017, real netto profit (taking into account gascost for extra e-production)
#daily optimum bidprice
fromtime = "2017-01-01"
totime = "2017-12-31"
bidvolume = 20
granularity = "day"

df102=R2plusMultiTimeOptimum(fromtime,totime,granularity,bidvolume,dfpricevol,dfR1R2cap,True,False)
remun102 = (int(df102.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df102.groupby(["timestamp"])[["returnCap"]].max().sum()))
PowerExtra102=df102.set_index("volumeActiv").groupby(["timestamp"])["returnAct"].idxmax().mean()*bidvolume*((pd.to_datetime(totime)+relativedelta(days=1))-(pd.to_datetime(fromtime))).days*24
bidav102=df102.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()
gascost102 = PowerExtra102 *2 * 17

print("average bidprice: " +str(float(bidav102)))
print("remuneration in €: " + str(remun102))
print("extra produced power in MWh: " +str(int(PowerExtra102)))
print("extra gascost in €: " + str(int(gascost102)))
print("real benefit in €: " + str(int(remun102-gascost102)))

In [ ]:
#compare remunerations and real benefits of optimising on quarter, hour, day and week
fromtime = "2017-07-03"
totime = "2017-07-09"

df100=R2plusMultiTimeOptimum(fromtime,totime,"quarter",20,dfpricevol,dfR1R2cap,True,False)
remun100 = (int(df100.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df100.groupby(["timestamp"])[["returnCap"]].max().sum()))

df101=R2plusMultiTimeOptimum(fromtime,totime,"hour",20,dfpricevol,dfR1R2cap,True,False)
remun101 = (int(df101.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df101.groupby(["timestamp"])[["returnCap"]].max().sum()))

df102=R2plusMultiTimeOptimum(fromtime,totime,"day",20,dfpricevol,dfR1R2cap,True,False)
remun102 = (int(df102.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df102.groupby(["timestamp"])[["returnCap"]].max().sum()))

df103=R2plusMultiTimeOptimum(fromtime,totime,"week",20,dfpricevol,dfR1R2cap,True,False)
remun103 = (int(df103.groupby(["timestamp"])[["returnAct"]].max().sum()) + int(df103.groupby(["timestamp"])[["returnCap"]].max().sum()))

PowerExtra100=df100["volumeActiv"].mean()*20*168
PowerExtra101=df101["volumeActiv"].mean()*20*168
PowerExtra102=df102["volumeActiv"].mean()*20*168
PowerExtra103=df103["volumeActiv"].mean()*20*168

bidav100=df100.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()
bidav101=df101.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()
bidav102=df102.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()
bidav103=df103.set_index("bidprice in euro/MWh").groupby(["timestamp"])[["returnAct"]].idxmax().mean()

#17€/MWh of gas on the spot market for october 2017, efficiency of electricity production of 50%
gascost100 = PowerExtra100 *2 * 17
gascost101 = PowerExtra101 *2 * 17
gascost102 = PowerExtra102 *2 * 17
gascost103 = PowerExtra103 *2 * 17

print("by quarter:")
print("average bidprice: " +str(float(bidav100)))
print("remuneration in €: " + str(remun100))
print("extra produced power in MWh: " +str(int(PowerExtra100)))
print("extra gascost in €: " + str(int(gascost100)))
print("real benefit in €: " + str(int(remun100-gascost100)))
print("")
print("by hour:")
print("average bidprice: " +str(float(bidav101)))
print("remuneration in €: " + str(remun101))
print("extra produced power in MWh: " +str(int(PowerExtra101)))
print("extra gascost in €: " + str(int(gascost101)))
print("real benefit in €: " + str(int(remun101-gascost101)))
print("")
print("by day:")
print("average bidprice: " +str(float(bidav102)))
print("remuneration in €: " + str(remun102))
print("extra produced power in MWh: " +str(int(PowerExtra102)))
print("extra gascost in €: " + str(int(gascost102)))
print("real benefit in €: " + str(int(remun102-gascost102)))
print("")
print("by week:")
print("average bidprice: " +str(float(bidav103)))
print("remuneration in €: " + str(remun103))
print("extra produced power in MWh: " +str(int(PowerExtra103)))
print("extra gascost in €: " + str(int(gascost103)))
print("real benefit in €: " + str(int(remun103-gascost103)))

In [ ]:
#overview of R1 and R2 capacity prices
fig, ax = plt.subplots()
dfmask.groupby(["reserve type","service type","country"])["average price in euro/MW/h"].plot(figsize=(16,6),legend=True, ax=ax,kind="line", style='.-',grid=True)
ax.set_xlabel("delivery period")
ax.set_ylabel("average price in euro/MW/h")
fig.suptitle("R1 and R2 capacity average prices")

In [ ]:
#overview of R1 and R2 capacity prices and volumes
fig, (ax,ax1) = plt.subplots(2,1,sharex=True)
dfmask.groupby(["reserve type","service type","country"])["total contracted volume in MW"].plot(figsize=(16,6),ax=ax,kind="line", style='.-',grid=True)
dfmask.groupby(["reserve type","service type","country"])["average price in euro/MW/h"].plot(figsize=(16,6),legend=True, ax=ax1,kind="line", style='.-',grid=True)
ax1.set_xlabel("delivery period")
ax1.set_ylabel("average price in euro/MW/h")
ax.set_ylabel("contracted volumes in MW")
fig.suptitle("R1 and R2 capacity average prices")

In [ ]:
#boxplots of R1 and R2 products


fig, (ax,ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,6)
dfmask.loc[dfmask["service type"] == "Symmetric200"]["average price in euro/MW/h"].plot(figsize=(16,10), showmeans=True, ax=ax,kind="box", showfliers=True)
ax.set_title("R1 Symmetric200")

dfmask.loc[dfmask["service type"] == "Symmetric100"]["average price in euro/MW/h"].plot(showmeans=True, ax=ax1,kind="box")
ax1.set_title("R1 Symmetric100")

dfmask.loc[(dfmask["service type"] == "Downward") & (dfmask["reserve type"] == "R1")]["average price in euro/MW/h"].plot(showmeans=True,ax=ax2,kind="box")
ax2.set_title("R1 Downward")

dfmask.loc[(dfmask["service type"] == "Upward") & (dfmask["reserve type"] == "R1")]["average price in euro/MW/h"].plot(showmeans=True,ax=ax3,kind="box")
ax3.set_title("R1 Upward")

dfmask.loc[(dfmask["service type"] == "Downward") & (dfmask["reserve type"] == "R2")]["average price in euro/MW/h"].plot(showmeans=True,ax=ax4,kind="box")
ax4.set_title("R2 Downward")

dfmask.loc[(dfmask["service type"] == "Upward") & (dfmask["reserve type"] == "R2")]["average price in euro/MW/h"].plot(showmeans=True,ax=ax5,kind="box")
ax5.set_title("R2 Upward")

fig.suptitle("R1 and R2 capacity average prices")

In [ ]:
################### define the optimum bidprice for the R2+ product ###########################
#based on historical data of both the capacity remuneration and activation remuneration of elia R2+ product, the optimal bidprice for a certain period is calculated.
#this is necessary as the pay-as-bid strategy is used to define the activation remuneration.

dfpricevol = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_03-2018.pkl")
dfcap = read_elia_cap("../data/Ancillary services, volumes and prices.xlsx")
dfsingle=R2plusSingleOptimum("2017-05-03","2017-05-03",1,dfpricevol,dfcap,True,True)

In [ ]:
################### define the optimum bidprices for the R2+ product ###########################
#based on historical data of both the capacity remuneration and activation remuneration of elia R2+ product, the optimal bidprices are calculated for a defined period of time.
#this is necessary as the pay-as-bid strategy is used to define the activation remuneration.

fromtime = "2017-05-01"
totime = "2017-05-07"
dfpricevol = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_03-2018.pkl")
dfcap = read_elia_cap("../data/Ancillary services, volumes and prices.xlsx")
dfmulti=R2plusMultiTimeOptimum(fromtime,totime,"day",10,dfpricevol,dfcap,True,True)

In [67]:
dfwind=pd.read_excel("../data/Generation_Produced_Historical_2018-09-September.xls",sheet_name="Sheet7")
dfwind=dfwind.dropna(how="all")

WARNING *** file size (376768) not 512 + multiple of sector size (512)


In [108]:
fromdate="01-2012"
todate="10-2012"

fromdatepd=pd.to_datetime(fromdate,format="%m-%Y")
todatepd=pd.to_datetime(todate,format="%m-%Y")
date = fromdatepd - relativedelta(months=1)


In [184]:
dfwind=pd.read_excel("http://publications.elia.be/Publications/Publications/FileRepository.v1.svc/DownloadFile?filePath=\Production_Data\Generation_Produced_Historical\Generation_Produced_Historical_"+str(date.year) +"-" + str(monthwithzero) + "-" + str(monthtext)+".XLS",sheet_name=gencat)
dfwind=dfwind.dropna(how="all")
dfwind.columns = dfwind.iloc[0]
dfwind=dfwind.reset_index()
dfwind=dfwind.drop("index",axis=1)
dfwind=dfwind.drop(0)
dfwind=dfwind.dropna(how="all",axis=1)
dfwind=dfwind.rename(columns={dfwind.columns[1]: "mm",dfwind.columns[2]: "yyyy"})
dfwind["Date"]= dfwind.apply(lambda row: datetime(int(row["yyyy"]),int(row["mm"]), int(row["dd"])),axis=1)
dfwind=dfwind.drop(["yyyy","mm","dd"],axis=1)
dfwind=dfwind.set_index("Date")
dfwind=dfwind.reset_index()



In [289]:
#fetch all production data from Elia and process

dfgen=pd.DataFrame()
dfgentotal=pd.DataFrame()

fromdate="09-2012"
todate="09-2018"

fromdatepd=pd.to_datetime(fromdate,format="%m-%Y")
todatepd=pd.to_datetime(todate,format="%m-%Y")
date = fromdatepd - relativedelta(months=1)

generation_categories=["nuclear","coal","fuel","gas","water","wind","other"]
dictmonths={"01":"January","02":"February","03":"March","04":"April","05":"May","06":"June","07":"July","08":"August","09":"September","10":"October","11":"November","12":"December"}
dictsheets={"total":"Sheet1","coal":"Sheet2","fuel":"Sheet3","gas":"Sheet4","nuclear":"Sheet5","water":"Sheet6","wind":"Sheet7","other":"Sheet8"}

while (date<todatepd):
    date=date+relativedelta(months=1)
    if date.month < 10:
        monthwithzero = "0" + str(date.month)
    else:
        monthwithzero=str(date.month)
        
    monthtext=dictmonths[monthwithzero]
  
    for i in range (0,len(generation_categories)):
        generation_category=generation_categories[i]
        gencat=dictsheets[generation_category]

        dfwind=pd.read_excel("http://publications.elia.be/Publications/Publications/FileRepository.v1.svc/DownloadFile?filePath=\Production_Data\Generation_Produced_Historical\Generation_Produced_Historical_"+str(date.year) +"-" + str(monthwithzero) + "-" + str(monthtext)+".XLS",sheet_name=gencat)
        dfwind=dfwind.dropna(how="all")
        dfwind.columns = dfwind.iloc[0]
        dfwind=dfwind.reset_index()
        dfwind=dfwind.drop("index",axis=1)
        dfwind=dfwind.drop(0)
        dfwind=dfwind.dropna(how="all",axis=1)
        dfwind=dfwind.rename(columns={dfwind.columns[1]: "mm",dfwind.columns[2]: "yyyy"})
        dfwind["Date"]= dfwind.apply(lambda row: datetime(int(row["yyyy"]),int(row["mm"]), int(row["dd"])),axis=1)
        dfwind=dfwind.drop(["yyyy","mm","dd"],axis=1)
        dfwind=dfwind.set_index("Date")
        dfwind=dfwind.reset_index()
        dfwind=pd.melt(dfwind,id_vars="Date",var_name="hour")
        dfwind.hour = dfwind.hour + ":00"
        dfwind["hour"]=pd.to_timedelta(dfwind["hour"])
        dfwind["Date"] = pd.to_datetime(dfwind["Date"]) + pd.to_timedelta(dfwind["hour"],unit="m")
        dfwind=dfwind.drop("hour",axis=1)
        dfwind=dfwind.set_index("Date")
        dfwind=dfwind.rename(columns={dfwind.columns[0]:generation_category})
        if i == 0:
            dfgen=dfwind
        if i > 0:
            dfgen = pd.concat([dfgen, dfwind], axis=1)
        
    dfgentotal=pd.concat([dfgentotal,dfgen],axis=0)
dfgentotal=dfgentotal.convert_objects(convert_numeric=True)
#water and wind has negative values sometimes, not possible with area plot
dfgentotal[dfgentotal < 0] = 0
#if zero, convert to np.nan
dfgentotal=dfgentotal.replace(0,np.nan)
#dropna: winter to summer hour --> 4 quarters of nan
dfgentotal=dfgentotal.dropna(how="all")
#CONVERT TO DATETIME
dfgentotal.index=(dfgentotal.index).to_pydatetime()


WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNING *** file size (384896) not 512 + multiple of sector size (512)
WARNIN

WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (368512) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (343936) not 512 + multiple of sector size (512)
WARNING *** file size (376704) not 512 + multiple of sector size (512)
WARNING *** file size (376704) not 512 + multiple of sector size (512)
WARNING *** file size (376704) not 512 + multiple of sector size (512)
WARNING *** file size (376704) not 512 + multiple of sector size (512)
WARNIN

WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNIN

WARNING *** file size (360384) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (384960) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNING *** file size (401344) not 512 + multiple of sector size (512)
WARNIN

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [216]:
dfgentotal.to_pickle("EliaProd012012to102018")

In [5]:
dfgentotal = pd.read_pickle("../data/EliaProd012012to102018.pkl")

In [81]:
fig,ax = plt.subplots()
dfgentotal.plot.area(ax=ax)

In [24]:
dfgentotalmask=dfgentotal.loc[(dfgentotal.index >= "2017-01-01") & (dfgentotal.index < "2018-01-01")]

In [25]:
dfgentotalmask=dfgentotalmask.replace(np.nan,0)

In [26]:
dfgentotalmask["wind"].sort_index().to_excel("output.xlsx")

In [27]:
dfbelpex=pd.read_excel("belpex2017.xlsx")

In [32]:
dfbelpex=dfbelpex.set_index("Date")

In [56]:
dfgentotalmask.index=dfgentotalmask.index.to_pydatetime()

In [62]:
dfbelpex["test"]=dfbelpex.index.to_pydatetime()

In [73]:
import plotly.graph_objs as go

The draw time for this plot will be slow for clients without much RAM.


C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [58]:
fig,(ax,ax1) = plt.subplots(2,1,sharex=True)
dfbelpex.plot(ax=ax,grid=True)
dfgentotalmask["wind"].sort_index().plot(ax=ax1,grid=True)

plotly_fig = tls.mpl_to_plotly( fig )
py.iplot(plotly_fig, filename='mpl-time-series')

data = [go.Scatter( x=dfbelpex.index, y=dfbelpex.Euro )]
py.iplot(data, filename='pandas-time-series')


C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\plotly\matplotlylib\renderer.py:390: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\plotly\matplotlylib\renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



The draw time for this plot will be slow for all clients.


C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [233]:
dfgentotal=dfgentotal.replace(np.nan,0)
dfgentotal["total"]=dfgentotal["nuclear"]+dfgentotal["coal"]+dfgentotal["fuel"]+dfgentotal["gas"]+dfgentotal["water"]+dfgentotal["wind"]+dfgentotal["other"]
dfgentotal=dfgentotal.replace(0,np.nan)

In [262]:
windperc=(dfgentotal["wind"]/dfgentotal["total"])

In [258]:
windperc=(dfgentotal["wind"]/dfgentotal["total"]).sort_values(ascending=False)
windperc=windperc.reset_index()
windperc=windperc.drop("Date",axis=1)
windperc.plot(grid=True)

ValueError: ordinal must be >= 1

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\tkinter\__init__.py", line 745, in callit
    func(*args)
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 323, in idle_draw
    self.draw()
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 304, in draw
    FigureCanvasAgg.draw(self)
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\matplotlib\backends\backend_agg.py", line 430, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\jensbaet\AppData\Local\conda\conda\envs\Elia\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwar

In [84]:
dfgen=pd.DataFrame()

year="2018"
month="10"
generation_categories=["coal","fuel","gas","nuclear","water","wind","other"]
dictmonths={"01":"January","02":"February","03":"March","04":"April","05":"May","06":"June","07":"July","08":"August","09":"September","10":"October","11":"November","12":"December"}
monthtext=dictmonths[month]
dictsheets={"total":"Sheet1","coal":"Sheet2","fuel":"Sheet3","gas":"Sheet4","nuclear":"Sheet5","water":"Sheet6","wind":"Sheet7","other":"Sheet8"}


for i in range (0,len(generation_categories)):
    generation_category=generation_categories[i]
    gencat=dictsheets[generation_category]

    dfwind=pd.read_excel("http://publications.elia.be/Publications/Publications/FileRepository.v1.svc/DownloadFile?filePath=\Production_Data\Generation_Produced_Historical\Generation_Produced_Historical_"+str(year) +"-" + str(month) + "-" + str(monthtext)+".XLS",sheet_name=gencat)
    dfwind=dfwind.dropna(how="all")
    dfwind.columns = dfwind.iloc[0]
    dfwind=dfwind.reset_index()
    dfwind=dfwind.drop("index",axis=1)
    dfwind=dfwind.drop(0)
    dfwind=dfwind.dropna(how="all",axis=1)
    dfwind=dfwind.rename(columns={dfwind.columns[1]: "mm",dfwind.columns[2]: "yyyy"})
    dfwind["Date"]= dfwind.apply(lambda row: datetime(int(row["yyyy"]),int(row["mm"]), int(row["dd"])),axis=1)
    dfwind=dfwind.drop(["yyyy","mm","dd"],axis=1)
    dfwind=dfwind.set_index("Date")
    dfwind=dfwind.reset_index()
    dfwind=pd.melt(dfwind,id_vars="Date",var_name="hour")
    dfwind.hour = dfwind.hour + ":00"
    dfwind["hour"]=pd.to_timedelta(dfwind["hour"])
    dfwind["Date"] = pd.to_datetime(dfwind["Date"]) + pd.to_timedelta(dfwind["hour"],unit="m")
    dfwind=dfwind.drop("hour",axis=1)
    dfwind=dfwind.set_index("Date")
    dfwind=dfwind.rename(columns={dfwind.columns[0]:generation_category})
    if i == 0:
        dfgen=dfwind
    if i > 0:
        dfgen = pd.concat([dfgen, dfwind], axis=1)

WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)
WARNING *** file size (221120) not 512 + multiple of sector size (512)


In [85]:
dfgen.plot()

In [54]:
#wind production data
dfwind=pd.read_excel("http://publications.elia.be/Publications/Publications/FileRepository.v1.svc/DownloadFile?filePath=\Production_Data\Generation_Produced_Historical\Generation_Produced_Historical_"+str(year) +"-" + str(month) + "-" + str(monthtext)+".XLS",sheet_name=gencat)
dfwind=dfwind.dropna(how="all")
dfwind.columns = dfwind.iloc[0]
dfwind=dfwind.reset_index()
dfwind=dfwind.drop("index",axis=1)
dfwind=dfwind.drop(0)
dfwind=dfwind.dropna(how="all",axis=1)
dfwind=dfwind.rename(columns={dfwind.columns[1]: "mm",dfwind.columns[2]: "yyyy"})
dfwind["Date"]= dfwind.apply(lambda row: datetime(int(row["yyyy"]),int(row["mm"]), int(row["dd"])),axis=1)
dfwind=dfwind.drop(["yyyy","mm","dd"],axis=1)
dfwind=dfwind.set_index("Date")
dfwind=dfwind.reset_index()
dfwind=pd.melt(dfwind,id_vars="Date",var_name="hour")
dfwind.hour = dfwind.hour + ":00"
dfwind["hour"]=pd.to_timedelta(dfwind["hour"])
dfwind["Date"] = pd.to_datetime(dfwind["Date"]) + pd.to_timedelta(dfwind["hour"],unit="m")
dfwind=dfwind.drop("hour",axis=1)
dfwind=dfwind.set_index("Date")
dfwind=dfwind.rename(columns={dfwind.columns[0]:generation_category})

WARNING *** file size (393088) not 512 + multiple of sector size (512)


In [37]:
dfwind1=dfwind.sort_values("value")
dfwind1=dfwind1.reset_index()
dfwind1=dfwind1.drop("Date",axis=1)

In [38]:
dfwind1.plot()